In [2]:
import os
import pickle
import torch
import numpy as np
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import transforms
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

#importing the cifar dataset.
def unpickle(file):
    with open(file, mode = "rb") as fil:
        try:
            temp_dict = pickle.load(fil, encoding = "bytes")
            return temp_dict
        except Exception as e:
            print(f"An error occured while loading: {e}")
            return None

mod_dict = {1:[],2:[],3:[],4:[],5:[]}
class data_prep:
    label_path = "/Users/Vedant Dutta/Desktop/CIFAR_10/batches.meta"
    root_path = "/Users/Vedant Dutta/Desktop/CIFAR_10"
    training_dict = {1:'data_batch_1', 2:'data_batch_2', 3:'data_batch_3', 4:'data_batch_4',5:'data_batch_5'}
    label_dict = unpickle(label_path)

    transformer = transforms.Compose([
                                      transforms.Resize((224,224)),
                                      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
                                      

    '''this is of the form, label_dict[0] = "Airplane"'''
    for num in tqdm(training_dict.keys()):
        new_path = os.path.join(root_path,training_dict[num])
        if not os.path.exists(new_path):
            print("error occured while joining paths for bactch num : {}".format(num))
        if unpickle(new_path) == None:
            print("Error loading the data")
        else:
            temp_data_dict = unpickle(new_path)
            '''the dict is of the following form : a 10,000X3072 numpy array, where for each row
            the first 1024 values are for channel1 and so on.
            ->The corresponding values of the dictionary are the labels for each of the pictures'''
            data_list = []
            labl_list = []
            data_array = np.array(temp_data_dict[b'data']).astype(np.float32) / 255.0
            for data, label_val in zip(data_array, temp_data_dict[b'labels']):
                d_tensor = torch.from_numpy(data).view(3, 32, 32)
                d_tensor_resized = transformer(d_tensor)  # Now you can apply transformer directly
                data_list.append(d_tensor_resized)
                labl_list.append(label_val)
            mod_dict[num] = (data_list, labl_list)
    
                
data_class = data_prep()
print(f"Loaded data for {len(mod_dict)} batches.")

for num, data in mod_dict.items():
    if data:  
        print(f"Batch {num}: {len(data[0][0])} images, {len(data[0][1])} labels.")
    else:
        print(f"Batch {num} is empty.")



100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:09<00:00, 25.87s/it]


Loaded data for 5 batches.
Batch 1: 3 images, 3 labels.
Batch 2: 3 images, 3 labels.
Batch 3: 3 images, 3 labels.
Batch 4: 3 images, 3 labels.
Batch 5: 3 images, 3 labels.


The data has now been prepared for each of the batches and it going to be used to train the VGGnet

In [3]:
class VGGNet(nn.Module):
    def __init__(self, num_classes=10):
        super(VGGNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU())
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU())
        self.layer4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer5 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer6 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU())
        self.layer7 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer8 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer9 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer10 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer11 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer12 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU())
        self.layer13 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(512 *7*7, 4096), 
            nn.ReLU())
        self.fc1 = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU())
        self.fc2 = nn.Linear(4096, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.layer9(out)
        out = self.layer10(out)
        out = self.layer11(out)
        out = self.layer12(out)
        out = self.layer13(out)
        out = out.view(out.size(0), -1)  # Flatten the output
        out = self.fc(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

In [ ]:
#training the model.
import torch.optim as optim

def aggregate_data(data_dict):
    train_data = []
    labl_arr = []
    for num in range(1,6):
        if data_dict[num]:
            data_list = data_dict[num][0]
            label_arr = data_dict[num][1]
            train_data.extend(data_list)
            labl_arr.extend(label_arr)   #this was initially append.
    return train_data, labl_arr

#we can now combine and then shuffle the data.
train_data,train_labels = aggregate_data(mod_dict)
indices = np.arange(len(train_data))
np.random.shuffle(indices)
shuffled_data = [train_data[i] for i in indices]
shuffled_labels = [train_labels[i] for i in indices]  #should be train_labels.

def training_func(model, data, num_epochs, optimizer, loss_func, batch_size):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss =0
        for i in tqdm(range(0, len(data[0]), batch_size)):
            images = data[0][i:i + batch_size]
            labels = data[1][i:i + batch_size]
            images = torch.stack(images)
            labels = torch.tensor(labels)
            optimizer.zero_grad()
            outputs = model(images)  #maybe forward should be removed.
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss / (len(data[0]) // batch_size):.4f}")

criterion = nn.CrossEntropyLoss()
model = VGGNet(num_classes = 10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
training_func(model, (shuffled_data, shuffled_labels), num_epochs=2, optimizer=optimizer, loss_func=criterion, batch_size=32)

    

 13%|██████████                                                                 | 211/1563 [1:17:31<7:05:39, 18.89s/it]

In [86]:
train_data = aggregate_data(mod_dict)
indices = np.arange(len(train_data[0]))
np.random.shuffle(indices)
shuffled_data = [train_data[0][i] for i in indices]
shuffled_labels = [train_data[1][i] for i in indices]
print(len(shuffled_data))

50000


In [124]:
print(len(train_data))

50000


In [123]:
print(len(train_labels))


2560256


In [8]:
def aggregate_data(data_dict):
    train_data = []
    labl_arr = []
    for num in range(1, 6):
        if data_dict[num]:
            data_list = data_dict[num][0]
            label_arr = data_dict[num][0]
            print(f"Batch {num}: {len(data_list)} images, {len(label_arr)} labels")  # Debugging line
            train_data.extend(data_list)
            labl_arr.extend(label_arr)
    print(f"Total training data: {len(train_data)}, Total labels: {len(labl_arr)}")  # Debugging line
    return train_data, labl_arr

# Aggregate data from batches
train_data, train_labels = aggregate_data(mod_dict)

# Check lengths
print(f"Number of training samples: {len(train_data)}")  # Should be 50000
print(f"Number of labels: {len(train_labels)}")  # Should also be 50000


Batch 1: 10000 images, 10000 labels
Batch 2: 10000 images, 10000 labels
Batch 3: 10000 images, 10000 labels
Batch 4: 10000 images, 10000 labels
Batch 5: 10000 images, 10000 labels
Total training data: 50000, Total labels: 50000
Number of training samples: 50000
Number of labels: 50000


In [7]:
print(len(mod_dict[5][1]))

10000
